In [1]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
EXCHANGE_RATE_API_KEY = userdata.get('EXCHANGE_RATE_API_KEY')

In [2]:
!pip install -q langchain-openai langchain-core requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.9 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

#### Create Tools

In [4]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

In [5]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  '''This function fetches the currency conversion factor between the given base currency and target currency'''
  url = f'https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{base_currency}/{target_currency}'
  response = requests.get(url)
  return response.json()

@tool
def convert(base_currency_value: int, conversion_factor: Annotated[float, InjectedToolArg]) -> float:
  '''This function converts the base currency value to the target currency'''
  return base_currency_value * conversion_factor

In [6]:
get_conversion_factor.invoke({'base_currency': 'USD', 'target_currency': 'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1759017601,
 'time_last_update_utc': 'Sun, 28 Sep 2025 00:00:01 +0000',
 'time_next_update_unix': 1759104001,
 'time_next_update_utc': 'Mon, 29 Sep 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 88.7384}

In [7]:
convert.invoke({'base_currency_value': 10, 'conversion_factor': 88.7384})

887.384

In [8]:
# tool binding
llm = ChatOpenAI(model_name='gpt-4o-mini')
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [9]:
messages = [HumanMessage('What is the conversion factor between USD and INR, Based on that can you convert 15 USD to INR')]

In [10]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, Based on that can you convert 15 USD to INR', additional_kwargs={}, response_metadata={})]

In [11]:
ai_message = llm_with_tools.invoke(messages)

In [12]:
messages.append(ai_message)

In [13]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_K1UPq0htea4LIYYumNHQ2JW5',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 15},
  'id': 'call_qI40aGyalkTpAc9QkgKcJPy6',
  'type': 'tool_call'}]

In [14]:
import json
from langchain_core.messages import ToolMessage

for tool_call in ai_message.tool_calls:
  # get the vakue if coversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call['args'])
    print(tool_message1)
    conversion_rate = tool_message1['conversion_rate']
    messages.append(ToolMessage(content=json.dumps(tool_message1), tool_call_id=tool_call['id']))

  if tool_call['name'] == 'convert':
      tool_call['args']['conversion_factor'] = conversion_rate
      tool_message2 = convert.invoke(tool_call['args'])
      print(tool_message2)
      messages.append(ToolMessage(content=str(tool_message2), tool_call_id=tool_call['id']))

{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1759017601, 'time_last_update_utc': 'Sun, 28 Sep 2025 00:00:01 +0000', 'time_next_update_unix': 1759104001, 'time_next_update_utc': 'Mon, 29 Sep 2025 00:00:01 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 88.7384}
1331.076


In [15]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, Based on that can you convert 15 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_K1UPq0htea4LIYYumNHQ2JW5', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_qI40aGyalkTpAc9QkgKcJPy6', 'function': {'arguments': '{"base_currency_value": 15}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 109, 'total_tokens': 162, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CKi1octao8IopNVDU0iGtgCcBvYgT', 'service_t

In [16]:
llm_with_tools.invoke(messages).content

'The conversion factor between USD and INR is approximately 88.7384. Therefore, converting 15 USD to INR results in approximately 1331.08 INR.'